In [43]:

# Import Libraries
import pandas as pd
import numpy as np
import os
from dask import dataframe as dd

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from dateutil.relativedelta import relativedelta

def ConvertDate(string):
    date = pd.to_datetime(string, format='%Y-%m-%d')
    return date


In [41]:

# Open datasets with flagged errors
Datasets = ['Dementia', 'Autism', 'Diabetes']

def OpenFile(FileName):
    Path = r"O:\GIRFT DiRAC\FlavienHardy\1_DataConsistency\Data\0_Raw\21.11"
    FilePath = os.path.join(Path, FileName)
    if ".csv" in FilePath:
        df_dataset = pd.read_csv(FilePath)
    else:
        df_dataset = pd.read_excel(FilePath)
    
    return df_dataset

FileName_Dementia = "PD dementia 04-11-21 (with ethnicity IMD and frailty).xlsx"
FileName_Autism = "Autism with IMD, HFRS and ethnicity 08-11-21.xlsx"
df_Dementia_ini = OpenFile(FileName_Dementia)
df_Autism_ini = OpenFile(FileName_Autism)

print(df_Dementia_ini.columns)
df_Dementia_ini.head()


Index(['P_Spell_ID1', 'Epikey', 'procedure_group', 'sex', 'EPIstart', 'EPIend',
       'HESID1', 'epiorder', 'ProvCode', 'Sitecode', 'LSOA_2011_Code',
       'main_specialty_code', 'Main_Specialty_Description', 'age_of_patient',
       'diagnosis_group', 'POD', 'MORT', 'FinY', 'Read30', 'Read90',
       'Admission_date1', 'Discharge_date1', 'Spell_Los', 'HFRS_Score',
       'HFRS_Band', 'Ethnicity'],
      dtype='object')


,P_Spell_ID1,Epikey,procedure_group,sex,EPIstart,EPIend,HESID1,epiorder,ProvCode,Sitecode,...,MORT,FinY,Read30,Read90,Admission_date1,Discharge_date1,Spell_Los,HFRS_Score,HFRS_Band,Ethnicity
0,1421,700925000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Male,2020-09-04,2020-09-05,00007773EC3BDCAF9441AFA32057315D,1,RTD,RTD02,...,0,2020/21,NaN,NaN,2020-09-04,2020-09-18,14,0.0,None,Not known
1,1421,700925000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Male,2020-09-05,2020-09-07,00007773EC3BDCAF9441AFA32057315D,2,RTD,RTD02,...,0,2020/21,NaN,NaN,2020-09-04,2020-09-18,14,0.0,None,Not known
2,1421,700925000000,U051~Y973~Y981~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Male,2020-09-07,2020-09-18,00007773EC3BDCAF9441AFA32057315D,3,RTD,RTD01,...,0,2020/21,NaN,NaN,2020-09-04,2020-09-18,14,0.0,None,Not known
3,8786,502700000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Female,2014-07-03,2014-07-12,0002CF863631188C9524321D4710EF31,1,RAJ,RDDH0,...,0,2014/15,NaN,NaN,2014-07-03,2014-07-17,14,0.0,None,White
4,8786,502700000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Female,2014-07-12,2014-07-17,0002CF863631188C9524321D4710EF31,2,RAJ,RDDH0,...,0,2014/15,NaN,NaN,2014-07-03,2014-07-17,14,0.0,None,White


# Pre-Processing

In [125]:

# Pre-process dataset: rename and reorganise features, convert date features
# --------------------------------------------------
def PreProcessing(df_ini, ConditionName):
    df = df_ini.copy()
    
    if ConditionName=='Diabetes':
        # Reorganising features
        ColumnList = ['P_Spell_ID1', 'HESID1',
                      'Admission_date1', 'Discharge_date1',
                      'EPIstart', 'EPIend', 'epiorder', 'Epikey',
                      'diagnosis_group', 'procedure_group', 
                      'age_of_patient', 'sex', 'Spell_Los', 'ProvCode', 'Sitecode',
                      'LSOA_2011_Code', 'Main_Specialty_Description', 'Main_Specialty_Description',
                      'POD', 'MORT', 'FinY', 'Read30', 'Read90']

        df = df.loc[:, ColumnList]
        
        DateFormat = '%Y-%m-%d %H:%M:%S'
        
    else:
        DateFormat = '%d/%m/%Y %H:%M'
    
    # Renaming features
    df = df.rename(columns={'P_Spell_ID1':'P_Spell_ID', 
                            'HESID1':'HESID',
                            'Admission_date1':'Admission_date',
                            'Discharge_date1':'Discharge_date',
                            'HFRS_score':'HFRS_Score',
                            'IMD_score':'IMD_Score',
                            'IMD_decile':'IMD_Decile'})
    
    DateFeatures = ['Admission_date', 'Discharge_date', 'EPIstart', 'EPIend']
    for date in DateFeatures:
        df.loc[:, date] = pd.to_datetime( df.loc[:, date], format=DateFormat )

    ColumnList = ['P_Spell_ID', 'HESID', 'LSOA_2011_Code',
                  'Admission_date', 'Discharge_date', 'EPIstart', 'EPIend', 'Epikey',
                  'diagnosis_group',
                  'Spell_Los', 'sex', 'age_of_patient',
                  'ProvCode', 'Main_Specialty_Description', 'MORT', 'Read30', 'Read90',
                  'HFRS_Score', 'HFRS_Band', 'Ethnicity']
    
    IMDList = ['IMD_Score', 'IMD_Decile']
    for col in IMDList:
        if col in df.columns:
            ColumnList = ColumnList + [col]

    df = df.loc[:, ColumnList]
    
    return df
# --------------------------------------------------

df_Dementia = PreProcessing(df_Dementia_ini, 'Dementia')
df_Autism = PreProcessing(df_Autism_ini, 'Autism')

print(df_Dementia.columns)
df_Dementia_ini.head()


Index(['P_Spell_ID', 'HESID', 'LSOA_2011_Code', 'Admission_date',
       'Discharge_date', 'EPIstart', 'EPIend', 'Epikey', 'diagnosis_group',
       'Spell_Los', 'sex', 'age_of_patient', 'ProvCode',
       'Main_Specialty_Description', 'MORT', 'Read30', 'Read90', 'HFRS_Score',
       'HFRS_Band', 'Ethnicity'],
      dtype='object')


,P_Spell_ID1,Epikey,procedure_group,sex,EPIstart,EPIend,HESID1,epiorder,ProvCode,Sitecode,...,MORT,FinY,Read30,Read90,Admission_date1,Discharge_date1,Spell_Los,HFRS_Score,HFRS_Band,Ethnicity
0,1421,700925000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Male,2020-09-04,2020-09-05,00007773EC3BDCAF9441AFA32057315D,1,RTD,RTD02,...,0,2020/21,NaN,NaN,2020-09-04,2020-09-18,14,0.0,None,Not known
1,1421,700925000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Male,2020-09-05,2020-09-07,00007773EC3BDCAF9441AFA32057315D,2,RTD,RTD02,...,0,2020/21,NaN,NaN,2020-09-04,2020-09-18,14,0.0,None,Not known
2,1421,700925000000,U051~Y973~Y981~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Male,2020-09-07,2020-09-18,00007773EC3BDCAF9441AFA32057315D,3,RTD,RTD01,...,0,2020/21,NaN,NaN,2020-09-04,2020-09-18,14,0.0,None,Not known
3,8786,502700000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Female,2014-07-03,2014-07-12,0002CF863631188C9524321D4710EF31,1,RAJ,RDDH0,...,0,2014/15,NaN,NaN,2014-07-03,2014-07-17,14,0.0,None,White
4,8786,502700000000,-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-...,Female,2014-07-12,2014-07-17,0002CF863631188C9524321D4710EF31,2,RAJ,RDDH0,...,0,2014/15,NaN,NaN,2014-07-03,2014-07-17,14,0.0,None,White


# Find Missing Mandatory Codes at Episode Level

In [126]:

# Find mentions of mandatory codes in CodeList in ICDfeature_Name
# --------------------------------------------------
def FindMandatoryCode(df_ini, ICDfeature_Name, ConditionName, CodeList):
    df = df_ini
    
    Codes = '|'.join(CodeList)

    # Look for ANY mention of mandatory code
    Filter = df[ICDfeature_Name].str.contains(Codes)
    df.loc[:, "ICD10_" + ConditionName] = np.where(Filter, 1, 0)

    return df
# --------------------------------------------------

# Mandatory codes for dementia
CodesDementia_Categories = "F00|F01|F02"
CodesDementia_Unspecified = "F03|F051"
Codes_Dementia_SubCategories = "F000|F001|F002|F009|F010|F011|F012|F013|F018|F019|F020|F021|F022|F023|F024|F028"
Codes_Dementia_Supp = "G301|G302|G308|G309"
CodesDementia = CodesDementia_Categories + "|" + CodesDementia_Unspecified + "|" + Codes_Dementia_SubCategories + "|" + Codes_Dementia_Supp
CodesDementia_List = CodesDementia.split("|")

# Mandatory codes for autism
CodesAutism_List = ['F84']

# Mandatory codes for diabetes
CodesDiabetes_List = ['E11']

# Look for mandatory codes
df_Dementia_2 = FindMandatoryCode(df_Dementia, "diagnosis_group", "Dementia", CodesDementia_List)
df_Autism_2 = FindMandatoryCode(df_Autism, "diagnosis_group", "Autism", CodesAutism_List)

print(df_Dementia_2.columns)
df_Dementia_2.head()


Index(['P_Spell_ID', 'HESID', 'LSOA_2011_Code', 'Admission_date',
       'Discharge_date', 'EPIstart', 'EPIend', 'Epikey', 'diagnosis_group',
       'Spell_Los', 'sex', 'age_of_patient', 'ProvCode',
       'Main_Specialty_Description', 'MORT', 'Read30', 'Read90', 'HFRS_Score',
       'HFRS_Band', 'Ethnicity', 'ICD10_Dementia'],
      dtype='object')


,P_Spell_ID,HESID,LSOA_2011_Code,Admission_date,Discharge_date,EPIstart,EPIend,Epikey,diagnosis_group,Spell_Los,...,age_of_patient,ProvCode,Main_Specialty_Description,MORT,Read30,Read90,HFRS_Score,HFRS_Band,Ethnicity,ICD10_Dementia
0,1421,00007773EC3BDCAF9441AFA32057315D,E01008446,2020-09-04,2020-09-18,2020-09-04,2020-09-05,700925000000,R410~E86X~R630~G20X~F067~G473~I679~D333~Z922~Z...,14,...,81,RTD,General Internal Medicine,0,NaN,NaN,0.0,None,Not known,0
1,1421,00007773EC3BDCAF9441AFA32057315D,E01008446,2020-09-04,2020-09-18,2020-09-05,2020-09-07,700925000000,R410~R441~E86X~R630~D529~E559~G20X~F067~I951~M...,14,...,81,RTD,Respiratory Medicine,0,NaN,NaN,0.0,None,Not known,0
2,1421,00007773EC3BDCAF9441AFA32057315D,E01008446,2020-09-04,2020-09-18,2020-09-07,2020-09-18,700925000000,G20X~F023~R410~R441~R630~D529~E559~I951~M549~G...,14,...,81,RTD,Geriatric Medicine,0,NaN,NaN,0.0,None,Not known,1
3,8786,0002CF863631188C9524321D4710EF31,E01021334,2014-07-03,2014-07-17,2014-07-03,2014-07-12,502700000000,N390~U898~N182~G20XD~F023A~K590~R33X~R600~R680...,14,...,88,RAJ,Geriatric medicine,0,NaN,NaN,0.0,None,White,1
4,8786,0002CF863631188C9524321D4710EF31,E01021334,2014-07-03,2014-07-17,2014-07-12,2014-07-17,502700000000,N390~U898~N182~G20XD~F023A~K590~R33X~R600~R680...,14,...,88,RAJ,Geriatric medicine,0,NaN,NaN,0.0,None,White,1


## Identify Coding Errors

In [127]:

# Remove episodes in first two years and at end of dataset + Extract first episodes
# --------------------------------------------------
def CleanUp_ExtremitiesDataset(df_ini, ConditionName):
    df = df_ini.copy()

    # Remove patients in first two years
    Filter_First2Years = df.loc[:, 'Discharge_date'] >= ConvertDate('2013-04-01')
    df = df.loc[Filter_First2Years, :]

    # Remove patients discharged after 2021-03-31
    Filter_EndDataset = df.loc[:, 'Discharge_date'] <= ConvertDate('2021-03-31')
    df = df.loc[Filter_EndDataset, :]

    # Find new first spell
    Filter_Code = df["ICD10_"+ConditionName] == 1
    df_Code = df.loc[Filter_Code, :]
    df_CodeFirst = df_Code.sort_values(by=['HESID', 'EPIstart', 'EPIend']).drop_duplicates(subset=['HESID'], keep='first')

    ColumnList = ['HESID', 'EPIstart', 'EPIend', 'Epikey',
                  'Admission_date', 'Discharge_date',
                  'ProvCode', 'Main_Specialty_Description']
    df = pd.merge( df, df_CodeFirst[ColumnList], how='left', on='HESID' )
    for col in df.columns:
        if "_x" in col:
            df = df.rename(columns={col:col[:-2]})
        elif "_y" in col:
            df = df.rename(columns={col:"FirstEpisode_"+col[:-2]})

    FirstDiagnosis = (df.loc[:, 'EPIstart']==df.loc[:, 'FirstEpisode_EPIstart']) & (df.loc[:, 'EPIend']==df.loc[:, 'FirstEpisode_EPIend'])
    df.loc[:, 'FirstDiagnosis'] = np.where(FirstDiagnosis, 1, 0)

    # Time difference between episode and first episode
    df.loc[:, 'TimeDiff_FirstEpisode'] = (df.loc[:, 'EPIstart'] - df.loc[:, 'FirstEpisode_EPIend']).dt.days

    # Remove all episodes without codes before first spell
    Filter_BeforeFirst = (df.loc[:, 'TimeDiff_FirstEpisode']<0) & (df.loc[:, 'FirstDiagnosis']==0)
    df = df.loc[~Filter_BeforeFirst, :]
    
    return df
# --------------------------------------------------

df_Dementia_3 = CleanUp_ExtremitiesDataset(df_Dementia_2, "Dementia")
df_Autism_3 = CleanUp_ExtremitiesDataset(df_Autism_2, "Autism")

print(df_Dementia_3.columns)
ColumnList = ['P_Spell_ID', 'HESID', 'EPIstart', 'EPIend', 'ICD10_Dementia', 'FirstEpisode_EPIstart', 'FirstDiagnosis']
df_Dementia_3.loc[:, :].head()


Index(['P_Spell_ID', 'HESID', 'LSOA_2011_Code', 'Admission_date',
       'Discharge_date', 'EPIstart', 'EPIend', 'Epikey', 'diagnosis_group',
       'Spell_Los', 'sex', 'age_of_patient', 'ProvCode',
       'Main_Specialty_Description', 'MORT', 'Read30', 'Read90', 'HFRS_Score',
       'HFRS_Band', 'Ethnicity', 'ICD10_Dementia', 'FirstEpisode_EPIstart',
       'FirstEpisode_EPIend', 'FirstEpisode_Epikey',
       'FirstEpisode_Admission_date', 'FirstEpisode_Discharge_date',
       'FirstEpisode_ProvCode', 'FirstEpisode_Main_Specialty_Description',
       'FirstDiagnosis', 'TimeDiff_FirstEpisode'],
      dtype='object')


,P_Spell_ID,HESID,LSOA_2011_Code,Admission_date,Discharge_date,EPIstart,EPIend,Epikey,diagnosis_group,Spell_Los,...,ICD10_Dementia,FirstEpisode_EPIstart,FirstEpisode_EPIend,FirstEpisode_Epikey,FirstEpisode_Admission_date,FirstEpisode_Discharge_date,FirstEpisode_ProvCode,FirstEpisode_Main_Specialty_Description,FirstDiagnosis,TimeDiff_FirstEpisode
2,1421,00007773EC3BDCAF9441AFA32057315D,E01008446,2020-09-04,2020-09-18,2020-09-07,2020-09-18,700925000000,G20X~F023~R410~R441~R630~D529~E559~I951~M549~G...,14,...,1,2020-09-07,2020-09-18,7.009250e+11,2020-09-04,2020-09-18,RTD,Geriatric Medicine,1,-11.0
3,8786,0002CF863631188C9524321D4710EF31,E01021334,2014-07-03,2014-07-17,2014-07-03,2014-07-12,502700000000,N390~U898~N182~G20XD~F023A~K590~R33X~R600~R680...,14,...,1,2014-07-03,2014-07-12,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,1,-9.0
4,8786,0002CF863631188C9524321D4710EF31,E01021334,2014-07-03,2014-07-17,2014-07-12,2014-07-17,502700000000,N390~U898~N182~G20XD~F023A~K590~R33X~R600~R680...,14,...,1,2014-07-03,2014-07-12,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,0,0.0
5,8787,0002CF863631188C9524321D4710EF31,E01021334,2014-12-15,2015-01-02,2014-12-15,2015-01-02,502700000000,J690~A419~N179~G20X~D696~I959~N390~E86X~R680~L...,18,...,0,2014-07-03,2014-07-12,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,0,156.0
6,8788,0002CF863631188C9524321D4710EF31,E01021334,2015-01-23,2015-01-26,2015-01-23,2015-01-26,502700000000,J181~N179~E86X~N390~T830~L890~G20X~I10X~M199~F...,3,...,0,2014-07-03,2014-07-12,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,0,195.0


In [128]:

# --------------------------------------------------
def FindErrors(df_ini, ConditionName):
    df = df_ini
    
    Filter_Errors = (df.loc[:, 'FirstDiagnosis']==0) & (df.loc[:, 'ICD10_'+ConditionName]==0)
    df.loc[:, 'Error_'+ConditionName] = np.where(Filter_Errors, 1, 0)

    df = df.sort_values(by=['HESID', 'EPIstart', 'EPIend'])
    
    return df
# --------------------------------------------------

df_Dementia_4 = FindErrors(df_Dementia_3, "Dementia")
df_Autism_4 = FindErrors(df_Autism_3, "Autism")

print(df_Dementia_4.columns)
ColumnList = ['P_Spell_ID', 'HESID', 'EPIstart', 'EPIend', 'ICD10_Dementia', 'FirstDiagnosis', 'Error_Dementia']
df_Dementia_4.loc[:, ColumnList].head(10)


Index(['P_Spell_ID', 'HESID', 'LSOA_2011_Code', 'Admission_date',
       'Discharge_date', 'EPIstart', 'EPIend', 'Epikey', 'diagnosis_group',
       'Spell_Los', 'sex', 'age_of_patient', 'ProvCode',
       'Main_Specialty_Description', 'MORT', 'Read30', 'Read90', 'HFRS_Score',
       'HFRS_Band', 'Ethnicity', 'ICD10_Dementia', 'FirstEpisode_EPIstart',
       'FirstEpisode_EPIend', 'FirstEpisode_Epikey',
       'FirstEpisode_Admission_date', 'FirstEpisode_Discharge_date',
       'FirstEpisode_ProvCode', 'FirstEpisode_Main_Specialty_Description',
       'FirstDiagnosis', 'TimeDiff_FirstEpisode', 'Error_Dementia'],
      dtype='object')


,P_Spell_ID,HESID,EPIstart,EPIend,ICD10_Dementia,FirstDiagnosis,Error_Dementia
2,1421,00007773EC3BDCAF9441AFA32057315D,2020-09-07,2020-09-18,1,1,0
3,8786,0002CF863631188C9524321D4710EF31,2014-07-03,2014-07-12,1,1,0
4,8786,0002CF863631188C9524321D4710EF31,2014-07-12,2014-07-17,1,0,0
5,8787,0002CF863631188C9524321D4710EF31,2014-12-15,2015-01-02,0,0,1
6,8788,0002CF863631188C9524321D4710EF31,2015-01-23,2015-01-26,0,0,1
7,18075,0005CFF4524423B83400095C9724194C,2017-06-25,2017-06-26,1,1,0
8,18075,0005CFF4524423B83400095C9724194C,2017-06-26,2017-06-30,1,0,0
9,18075,0005CFF4524423B83400095C9724194C,2017-06-30,2017-07-18,1,0,0
13,27150,0007D22296166FEAC8E4E85FEAED6EF0,2020-01-03,2020-02-03,1,1,0
14,27151,0007D22296166FEAC8E4E85FEAED6EF0,2020-02-03,2020-05-05,0,0,1


# Count and Characterise Errors at Episode Level

In [130]:

# Count missing mandatory codes at spell level
# --------------------------------------------------
def EpisodeLevel_CountErrors(df_ini, ConditionName):
    df = df_ini
    ErrorName = "Error_"+ConditionName
    
    # Find subsequent episodes
    Filter_SubsequentEpisodes = (df.loc[:, 'FirstDiagnosis']==0)
    df_SubsequentEpisodes = df.loc[Filter_SubsequentEpisodes, :]

    Sum = df_SubsequentEpisodes.loc[:, ErrorName].sum()
    Count = df_SubsequentEpisodes.loc[:, ErrorName].shape[0]
    Prop = 100 * Sum / Count
    
    print('-----------------------------')
    print(ConditionName.upper())
    print('Number of errors:', Sum)
    print('Number of subsequent EPISODES where mandatory codes should be used:', Count)
    print('Proportion of errors: %0.2f' % Prop, '%')
    
    return Sum, Count, Prop
# --------------------------------------------------

Sum_Dementia, Count_Dementia, Prop_Dementia = EpisodeLevel_CountErrors(df_Dementia_4, 'Dementia')
Sum_Autism, Count_Autism, Prop_Autism = EpisodeLevel_CountErrors(df_Autism_4, 'Autism')


-----------------------------
DEMENTIA
Number of errors: 33917
Number of subsequent EPISODES where mandatory codes should be used: 133985
Proportion of errors: 25.31 %
-----------------------------
AUTISM
Number of errors: 222595
Number of subsequent EPISODES where mandatory codes should be used: 507465
Proportion of errors: 43.86 %


In [131]:

# Merge IMD data with dementia dataset if necessary
# --------------------------------------------------
def LinkIMD(df_ini, df_IMD):
    
    ColumnList = ['LSOA code (2011)', 'Index of Multiple Deprivation (IMD) Score',
                  'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)']
    df_IMD_Sub = df_IMD.loc[:, ColumnList].rename(columns={'LSOA code (2011)':'LSOA_2011_Code',
                                                           'Index of Multiple Deprivation (IMD) Score':'IMD_Score',
                                                           'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)':'IMD_Decile'})
    df_IMD_Sub = df_IMD_Sub.drop_duplicates(subset=['LSOA_2011_Code'], keep='first')

    df_MergeIMD = pd.merge(df_ini, df_IMD_Sub, on='LSOA_2011_Code', how='left')

    return df_MergeIMD
# --------------------------------------------------

Dementia_ManuallyLinkIMD = True

if Dementia_ManuallyLinkIMD:
    print('Importing IMD dataset...')
    # IMD scores
    Path_IMD = r"O:\GIRFT DiRAC\FlavienHardy\1_DataConsistency\Data\0_Raw"
    FileName_IMD = "File_7_ID_2015_All_ranks__deciles_and_scores_for_the_Indices_of_Deprivation__and_population_denominators.csv"
    FilePath_IMD = os.path.join(Path_IMD, FileName_IMD)
    df_IMD = pd.read_csv(FilePath_IMD)
    print('Merging...')
    df_Dementia_IMD = LinkIMD(df_Dementia_4, df_IMD)

df_Autism_IMD = df_Autism_4

print(df_Dementia_IMD.columns)
df_Dementia_IMD.head()



Importing IMD dataset...
Merging...
Index(['P_Spell_ID', 'HESID', 'LSOA_2011_Code', 'Admission_date',
       'Discharge_date', 'EPIstart', 'EPIend', 'Epikey', 'diagnosis_group',
       'Spell_Los', 'sex', 'age_of_patient', 'ProvCode',
       'Main_Specialty_Description', 'MORT', 'Read30', 'Read90', 'HFRS_Score',
       'HFRS_Band', 'Ethnicity', 'ICD10_Dementia', 'FirstEpisode_EPIstart',
       'FirstEpisode_EPIend', 'FirstEpisode_Epikey',
       'FirstEpisode_Admission_date', 'FirstEpisode_Discharge_date',
       'FirstEpisode_ProvCode', 'FirstEpisode_Main_Specialty_Description',
       'FirstDiagnosis', 'TimeDiff_FirstEpisode', 'Error_Dementia',
       'IMD_Score', 'IMD_Decile'],
      dtype='object')


,P_Spell_ID,HESID,LSOA_2011_Code,Admission_date,Discharge_date,EPIstart,EPIend,Epikey,diagnosis_group,Spell_Los,...,FirstEpisode_Epikey,FirstEpisode_Admission_date,FirstEpisode_Discharge_date,FirstEpisode_ProvCode,FirstEpisode_Main_Specialty_Description,FirstDiagnosis,TimeDiff_FirstEpisode,Error_Dementia,IMD_Score,IMD_Decile
0,1421,00007773EC3BDCAF9441AFA32057315D,E01008446,2020-09-04,2020-09-18,2020-09-07,2020-09-18,700925000000,G20X~F023~R410~R441~R630~D529~E559~I951~M549~G...,14,...,7.009250e+11,2020-09-04,2020-09-18,RTD,Geriatric Medicine,1,-11.0,0,7.717,9.0
1,8786,0002CF863631188C9524321D4710EF31,E01021334,2014-07-03,2014-07-17,2014-07-03,2014-07-12,502700000000,N390~U898~N182~G20XD~F023A~K590~R33X~R600~R680...,14,...,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,1,-9.0,0,14.370,6.0
2,8786,0002CF863631188C9524321D4710EF31,E01021334,2014-07-03,2014-07-17,2014-07-12,2014-07-17,502700000000,N390~U898~N182~G20XD~F023A~K590~R33X~R600~R680...,14,...,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,0,0.0,0,14.370,6.0
3,8787,0002CF863631188C9524321D4710EF31,E01021334,2014-12-15,2015-01-02,2014-12-15,2015-01-02,502700000000,J690~A419~N179~G20X~D696~I959~N390~E86X~R680~L...,18,...,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,0,156.0,1,14.370,6.0
4,8788,0002CF863631188C9524321D4710EF31,E01021334,2015-01-23,2015-01-26,2015-01-23,2015-01-26,502700000000,J181~N179~E86X~N390~T830~L890~G20X~I10X~M199~F...,3,...,5.027000e+11,2014-07-03,2014-07-17,RAJ,Geriatric medicine,0,195.0,1,14.370,6.0


In [132]:

# --------------------------------------------------
def ErrorCharacteristics(df_ini, ConditionName):
    df = df_ini.copy()
    ErrorName = 'Error_'+ConditionName
    
    Filter_SubsequentEpisodes = (df.loc[:, 'FirstDiagnosis']==0)
    df = df.loc[Filter_SubsequentEpisodes, :]

    Filter = (df.loc[:, ErrorName]==1)
    df_Errors = df.loc[Filter, :]
    df_NoErrors = df.loc[~Filter, :]
    
    print('Number of errors at spell level:', df_Errors.shape[0])
    print('Total number of subsequent spells:', df.shape[0])
    print('Proportion of errors: %0.2f' % (100*df_Errors.shape[0]/df.shape[0]), '%')

    # --------------------------------------------
    # Errors by age bands
    AgeMin_List = [0, 18, 40, 60, 80] + [999]
    AgeMax_List = [age-1 for age in AgeMin_List[1:] ] + [999]
    for k in range(len(AgeMin_List)):
        AgeMin = AgeMin_List[k]
        AgeMax = AgeMax_List[k]
        FilterAge = (df['age_of_patient']>=AgeMin) & (df['age_of_patient']<=AgeMax)

        df_Age = df.loc[FilterAge, :]
        Sum = df_Age.loc[:, ErrorName].sum()
        Count = df_Age.loc[:, ErrorName].count()
        Prop_Errors = 100 * Sum/Count
        print('Age band:', AgeMin, '-', AgeMax)
        print('\t Number of errors:', Sum, 'out of', Count, ', corresponding to %0.2f' % Prop_Errors, '%')

    print('\n')

    # --------------------------------------------
    # Errors by sex
    List = ['Female', 'Male']
    for sex in List:
        Filter_sex = (df.loc[:, 'sex']==sex)
        df_sex =  df.loc[Filter_sex, :]
        Sum = df_sex.loc[:, ErrorName].sum()
        Count = df_sex.loc[:, ErrorName].count()
        Prop_Errors = 100*Sum/Count
        print('Sex:', sex)
        print('\t Number of errors:', Sum, 'out of', Count, ', corresponding to %0.2f' % Prop_Errors, '%')

    SexString = '|'.join(List)
    Filter_Sex = df.loc[:, 'sex'].str.contains(SexString)
    df_MissingSex = df.loc[~Filter_Sex, :]
    Sum = df_MissingSex.loc[:, ErrorName].sum()
    Count = df_MissingSex.loc[:, ErrorName].count()
    Prop_Errors = 100*Sum/Count
    print('Sex:', 'Other/Missing')
    print('\t Number of errors:', Sum, 'out of', Count, ', corresponding to %0.2f' % Prop_Errors, '%')
    print('\n')
    
    print('\n')
    # --------------------------------------------
    # Ethnicity
    if 'Ethnicity' in df.columns:
        df.loc[:, 'Ethnicity'] = df.loc[:, 'Ethnicity'].fillna('not known')
        df.loc[:, 'Ethnicity'] = df.loc[:, 'Ethnicity'].str.lower()
        
        EthnicityList = [ x.lower() for x in set(df.loc[:, 'Ethnicity'].values) ]
#         Count = df.shape[0]
        for ethnicity in EthnicityList:
            Filter = df.loc[:, 'Ethnicity']==ethnicity
            Sum = df.loc[Filter, ErrorName].sum()
            Count = df.loc[Filter, ErrorName].shape[0]
            Prop = 100*Sum/Count
            print('Number of', ethnicity, ' :', Sum, 'corresponding to %0.2f' % Prop, '%')

    print('\n')

    if 'IMD_Decile' in df.columns:
        Decile_List = range(1,10+1)
        for k in range(5):
            Decile_Max = Decile_List[k]*2
            Decile_Min = Decile_Max-1

            Filter_Quintiles = (df["IMD_Decile"]>=Decile_Min) & (df["IMD_Decile"]<=Decile_Max)
            Sum = df.loc[Filter_Quintiles, ErrorName].sum()
            Count = df.loc[Filter_Quintiles, ErrorName].count()
            Prop_Errors = 100*Sum/Count
            print('IMD Deciles:', Decile_Min, '-', Decile_Max)
            print('\t Number of errors:', Sum, 'out of', Count, ', corresponding to %0.2f' % Prop_Errors, '%')
# --------------------------------------------------


Conditions = ['Dementia', 'Autism', 'Diabetes']
# ErrorCharacteristics(df_Dementia_IMD, Conditions[0])
ErrorCharacteristics(df_Autism_IMD, Conditions[1])
# ErrorCharacteristics(df_diabetes_ini, Conditions[2])


Number of errors at spell level: 222595
Total number of subsequent spells: 507465
Proportion of errors: 43.86 %
Age band: 0 - 17
	 Number of errors: 64944 out of 181878 , corresponding to 35.71 %
Age band: 18 - 39
	 Number of errors: 95602 out of 204396 , corresponding to 46.77 %
Age band: 40 - 59
	 Number of errors: 38679 out of 75811 , corresponding to 51.02 %
Age band: 60 - 79
	 Number of errors: 20203 out of 40005 , corresponding to 50.50 %
Age band: 80 - 998
	 Number of errors: 3126 out of 5290 , corresponding to 59.09 %
Age band: 999 - 999
	 Number of errors: 41 out of 85 , corresponding to 48.24 %


Sex: Female
	 Number of errors: 85029 out of 180557 , corresponding to 47.09 %
Sex: Male
	 Number of errors: 137566 out of 326908 , corresponding to 42.08 %


<ipython-input-132-191b8e98c825>:52: RuntimeWarning: invalid value encountered in longlong_scalars
  Prop_Errors = 100*Sum/Count


Sex: Other/Missing
	 Number of errors: 0 out of 0 , corresponding to nan %




Number of not known  : 43305 corresponding to 41.88 %
Number of other ethnic groups  : 12388 corresponding to 40.87 %
Number of asian or asian british  : 6147 corresponding to 37.96 %
Number of white  : 152666 corresponding to 45.28 %
Number of mixed  : 3044 corresponding to 38.40 %
Number of black or black british  : 5045 corresponding to 40.49 %


IMD Deciles: 1 - 2
	 Number of errors: 65004 out of 144730 , corresponding to 44.91 %
IMD Deciles: 3 - 4
	 Number of errors: 53205 out of 117975 , corresponding to 45.10 %
IMD Deciles: 5 - 6
	 Number of errors: 40848 out of 96722 , corresponding to 42.23 %
IMD Deciles: 7 - 8
	 Number of errors: 33698 out of 78717 , corresponding to 42.81 %
IMD Deciles: 9 - 10
	 Number of errors: 26634 out of 63062 , corresponding to 42.23 %


In [134]:

SaveCSV = True
if SaveCSV:
    df_Dementia_IMD.to_csv(r"O:\GIRFT DiRAC\FlavienHardy\1_DataConsistency\Data\1_Processed\DementiaErrors_EpisodeLevel.csv")
    df_Autism_IMD.to_csv(r"O:\GIRFT DiRAC\FlavienHardy\1_DataConsistency\Data\1_Processed\AutismErrors_EpisodeLevel.csv")
